# IaaS Linux Logs Review Template - Splunk
BTV Project Obsidian, 2023

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

This playbook is to help validating available logs
It helps to baseline environment, identify gaps and control points.

It targets Defcon31 BlueTeamVillage Project obsidian environment and splunk platform but it can be adapt to other logging platforms.

Possible sources
* systemd, journald, su, sshd, sudo, cron, at
* auditd
* osquery
* sysmonforlinux

External sources like EDR or network are intentionally not covered here.

## Resources

* https://github.com/microsoft/msticpy/
* https://infosecjupyterthon.com/
* https://dropbox.tech/security/how-dropbox-security-builds-better-tools-for-threat-detection-and-incident-response
* https://github.com/SigmaHQ/sigma
* cli usage
  * as is `jupyter run notebook.ipynb --allow-errors` - https://docs.jupyter.org/en/latest/running.html#using-a-command-line-interface
  * with parameters `papermill input.ipynb output.ipynb -p alpha 0.6 -p l1_ratio 0.1` - https://papermill.readthedocs.io/en/latest/usage-workflow.html 

(Linux)
* https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/Entity%20Explorer%20-%20Linux%20Host.ipynb
* https://securitydatasets.com/notebooks/atomic/linux/intro.html

## Findings

_Put your findings here_

## Table of Contents

* Import
* Configuration
* Queries
  * Authentication
    * fail
    * success
  * Remote access
  * Privilege escalation
  * Privileged users activities? root, Administrator...
  * Service activities
    * Time
    * Logging
    * Scheduled tasks
  * System boot, on/off
  * Process activities
  * Network activities
  * File Integrity Monitoring (FIM)
  * Auditd
  * AV logs?
  * Web logs?
  * Misc

## Import

In [ ]:
# Check we are running Python 3.6
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [ ]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime,timedelta
import os

In [ ]:
# path to config file
os.environ['MSTICPYCONFIG'] = '/home/ubuntu/msticpyconfig.yaml'
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
print('Imports Complete')

## Configuration

In [ ]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls
mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [ ]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 4, 29), units='days', max_before=3, before=1, max_after=0)
q_times.display()

In [ ]:
query_common_args = '''agent.name=ip-172-16-40-100 earliest="04/29/2023:01:00:00" latest="04/29/2023:23:00:00"'''

In [ ]:
# Configuration
# if free splunk, 
#  * enable the 'allowRemoteLogin' setting in your server.conf file - /opt/splunk/etc/system/local/server.conf
splunk_prov = QueryProvider('Splunk')
splunk_prov.connect()

In [ ]:
# pandas
pd.set_option('display.max_colwidth', 500)

## Queries

### Timeperiod

Let's confirm that we have logs for the targeted timeperiod.

In [ ]:
q_times.start

In [ ]:
q_times.end

In [ ]:
query = f'''Syslog {query_common_args}
| summarize max(TimeGenerated),min(TimeGenerated)
'''
df_timeperiod = qry_prov.exec_query(query)
df_timeperiod.head(results_limit)

### Authentication

In [ ]:
splunk_query = f'''search index=linux su: {query_common_args}
| stats count by host,message'''
df_su = splunk_prov.exec_query(splunk_query)
df_su.head(10)

In [ ]:
splunk_query = f'''search index=linux session {query_common_args}
| stats count by host,message'''
df_session = splunk_prov.exec_query(splunk_query)
df_session.head(10)

### Remote access

In [ ]:
splunk_query = f'''search index=linux sshd (accepted OR failed) {query_common_args}
| stats count by host,message'''
df_ssh = splunk_prov.exec_query(splunk_query)

In [ ]:
df_ssh[['host', 'message']].head(10)
# df_ssh.head(10)

In [ ]:
# sshd[1234]: Starting session: subsystem 'sftp' for USER from 10.x.y.z port 55098 id 1
splunk_query = f'''search index=linux sshd subsystem {query_common_args}
| stats count by host,message'''
df_sftp = splunk_prov.exec_query(splunk_query)

In [ ]:
splunk_query = f'''search index=osquery {query_common_args}
| spath input=message
| search name="pack_osquery-custom-pack_authorized_keys"'''
df_ssh_authorized_keys = splunk_prov.exec_query(splunk_query)
df_ssh_authorized_keys.head(10)

### Privilege Escalation

In [ ]:
splunk_query = f'''search index=linux sudo {query_common_args}
| stats count by host,message'''
df_sudo = splunk_prov.exec_query(splunk_query)
df_sudo.head(10)

### Services activities

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (start OR stop OR Deactivated) {query_common_args}
| stats count by host,message'''
df_services = splunk_prov.exec_query(splunk_query)
df_services.head(10)

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (ntpd OR openntpd OR ntpdate OR rdate OR chrony) {query_common_args}
| stats count by host,message'''
df_service_time = splunk_prov.exec_query(splunk_query)
df_service_time.head(10)

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (journald OR rsyslog OR syslog-ng) {query_common_args}
| stats count by host,message'''
df_service_logging = splunk_prov.exec_query(splunk_query)
df_service_logging.head(10)

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (cron OR at OR systemd-timers) {query_common_args}
| stats count by host,message'''
df_service_scheduledtasks = splunk_prov.exec_query(splunk_query)
df_service_scheduledtasks.head(10)

In [ ]:
splunk_query = f'''search index=linux "input.type"!=journald (start OR stop OR Deactivated) {query_common_args}
| stats count by host,agent.type,input.type,message'''
df_services2 = splunk_prov.exec_query(splunk_query)
df_services2.head(10)

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd systemd-resolved {query_common_args}
| stats count by host,agent.type,input.type,message'''
df_systemdresolved1 = splunk_prov.exec_query(splunk_query)
df_systemdresolved1.head(10)

In [ ]:
splunk_query = f'''search index=linux systemd-resolved {query_common_args}
| stats count by host,agent.type,log.file.path,message'''
df_systemdresolved2 = splunk_prov.exec_query(splunk_query)
df_systemdresolved2.head(10)

### System boot, on/off

In [ ]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (halt OR shutdown???) {query_common_args}
| stats count by host,message'''
df_system_onoff = splunk_prov.exec_query(splunk_query)
if not df_system_onoff or df_system_onoff.empty:
    print("No results")
else:
    df_system_onoff.head(10)

### Error, warnings

In [ ]:
splunk_query = f'''search index=linux error {query_common_args}
| head 10'''
df_errors = splunk_prov.exec_query(splunk_query)
df_errors.head(5)

In [ ]:
splunk_query = f'''search index=linux warn {query_common_args}
| head 10'''
df_warn = splunk_prov.exec_query(splunk_query)
df_warn.head(5)

### Process activities

In [ ]:
splunk_query = f'''search index=sysmonforlinux {query_common_args}
| stats count by RuleName,User,Image,CommandLine,ParentCommandLine'''
df_process = splunk_prov.exec_query(splunk_query)
df_process.head(10)

In [ ]:
splunk_query = f'''search index=sysmonforlinux {query_common_args} (kebrute OR crackmapexec OR curl OR wget OR pip3 OR git)
| stats count by RuleName,User,Image,CommandLine,ParentCommandLine'''
df_process2 = splunk_prov.exec_query(splunk_query)
df_process2.head(20)

In [ ]:
splunk_query = f'''search index=sysmonforlinux {query_common_args} (kebrute OR crackmapexec OR curl OR wget OR pip3 OR git OR 11317)
| stats count by host,RuleName,UtcTime,User,Image,ProcessId,CommandLine,ParentProcessId,ParentCommandLine'''
df_process_tree = splunk_prov.exec_query(splunk_query)

In [ ]:
df_process_tree.head(5)

In [ ]:
df_process_tree[df_process_tree['ProcessId'] == 11317]

In [ ]:
# Process tree
from msticpy.transform.proc_tree_builder import LX_EVENT_SCH
from copy import copy
cust_lx_schema = copy(LX_EVENT_SCH)

cust_lx_schema.time_stamp = "UtcTime"
cust_lx_schema.host_name_column = "host"
cust_lx_schema.cmd_line = "CommandLine"
cust_lx_schema.process_name = "Image"
cust_lx_schema.process_id = "ProcessId"
cust_lx_schema.parent_id = "ParentProcessId"
cust_lx_schema.user_name = "User"
cust_lx_schema.event_id_column = None
cust_lx_schema.event_id_identifier = None

# now supply the schema as the schema parameter
#process_tree.build_process_tree(df_process_tree, schema=cust_lx_schema)
df_process_tree.mp_plot.process_tree(schema=cust_lx_schema)

In [ ]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_processes {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_outbound_connections"
| stats count by name,action,columns.pid,columns.cmdline,columns.ppid,columns.pcmdline'''
df_process9 = splunk_prov.exec_query(splunk_query)
if not df_process9 or df_process9.empty:
    print("No results")
else:
    df_process9.head(10)

### Network activities

In [ ]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_dns_resolvers {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_dns_resolvers"
| stats count by name,action,columns.address,columns.type'''
df_dns = splunk_prov.exec_query(splunk_query)

In [ ]:
 df_dns.head(10)

In [ ]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_outbound_connections {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_outbound_connections"
| stats count by name,action,columns.username,columns.name,columns.path,columns.cmdline,columns.remote_address'''
df_outbound = splunk_prov.exec_query(splunk_query)

In [ ]:
if not df_outbound or df_outbound.empty:
    print("No results")
else:
    df_outbound.head(10)

### File Integrity Monitoring

In [ ]:
splunk_query = f'''search index=osquery fim {query_common_args}
| spath input=message
| where name="fim"
| stats count by name,columns.target_path,columns.action'''
df_fim = splunk_prov.exec_query(splunk_query)
if not df_fim or df_fim.empty:
    print("No results")
else:
    df_fim.head(10)

### Auditd

by type, keywords, exe

### Selinux

### Local AV, EDR

### Misc

In [ ]:
splunk_prov.list_queries()

In [ ]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_python_packages {query_common_args}
| search name="pack_osquery-custom-pack_python_packages"
| stats count by name,action,columns.name,columns.summary,columns.version'''
df_python = splunk_prov.exec_query(splunk_query)

In [ ]:
if not df_python or df_python.empty:
    print("No results")
else:
    df_python.head(10)